<a href="https://colab.research.google.com/github/FaizFP/Dicoding--Analystic-Sentiments/blob/main/Scraping_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install google-play-scraper
import pandas as pd
from google_play_scraper import reviews_all,reviews, Sort
!pip install nltk sastrawi
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re


In [27]:
APP_ID = "com.gojek.app"

In [28]:
# Mengambil 11000 ulasan TERBARU.
# Kita gunakan reviews(), bukan reviews_all(), agar bisa dibatasi.
result, continuation_token = reviews(
    APP_ID,
    lang='id',
    country='id',
    sort=Sort.NEWEST,  # Opsi: mengambil ulasan yang paling baru
    count=11000,       # Batasi jumlah ulasan yang diambil
    filter_score_with=None # Ambil semua skor agar bisa kita labeli nanti
)

# Mengubah hasil scraping menjadi DataFrame pandas
df_reviews = pd.DataFrame(result)

# Menampilkan informasi hasil scrape
print(f"Berhasil mengambil {len(df_reviews)} ulasan.")
print(df_reviews.head())

Berhasil mengambil 11000 ulasan.
                               reviewId         userName  \
0  93b5e247-748c-4102-a208-eaa4ade74b72  Pengguna Google   
1  c6827243-0d0b-4ed1-b92d-311f81a1f362  Pengguna Google   
2  0bf5c1f0-4a7f-41aa-aa5c-242c24ce000e  Pengguna Google   
3  85b6521a-c57d-42de-ade1-3ea1e63abfc3  Pengguna Google   
4  286fbfb4-3f59-4b70-b390-978659dfdb79  Pengguna Google   

                                           userImage  \
0  https://play-lh.googleusercontent.com/EGemoI2N...   
1  https://play-lh.googleusercontent.com/EGemoI2N...   
2  https://play-lh.googleusercontent.com/EGemoI2N...   
3  https://play-lh.googleusercontent.com/EGemoI2N...   
4  https://play-lh.googleusercontent.com/EGemoI2N...   

                                             content  score  thumbsUpCount  \
0                                                 ok      5              0   
1  Gabisa kirim ulasan makanan, ga ada opsi send ...      1              0   
2                                  

In [29]:
df_reviews.head(5)

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,93b5e247-748c-4102-a208-eaa4ade74b72,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,ok,5,0,None,2025-09-28 13:53:21,None,NaT,None
1,c6827243-0d0b-4ed1-b92d-311f81a1f362,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Gabisa kirim ulasan makanan, ga ada opsi send ...",1,0,5.33.1,2025-09-28 13:37:42,"Hai Kak Erdio, mohon maaf atas kendalanya. Pas...",2025-09-28 15:29:24,5.33.1
2,0bf5c1f0-4a7f-41aa-aa5c-242c24ce000e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,baik dan ramah,5,0,5.30.1,2025-09-28 13:37:08,None,NaT,5.30.1
3,85b6521a-c57d-42de-ade1-3ea1e63abfc3,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,baru kali ini gw kesulitan batalkan tiket krl ...,1,0,5.33.1,2025-09-28 13:26:14,"Hai Kak Mawardi, mohon maaf atas ketidaknyaman...",2025-09-28 15:27:56,5.33.1
4,286fbfb4-3f59-4b70-b390-978659dfdb79,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Alhamdulillah sudah bantu,4,0,5.33.1,2025-09-28 13:03:25,None,NaT,5.33.1


In [30]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              11000 non-null  object        
 1   userName              11000 non-null  object        
 2   userImage             11000 non-null  object        
 3   content               11000 non-null  object        
 4   score                 11000 non-null  int64         
 5   thumbsUpCount         11000 non-null  int64         
 6   reviewCreatedVersion  8550 non-null   object        
 7   at                    11000 non-null  datetime64[ns]
 8   replyContent          3054 non-null   object        
 9   repliedAt             3054 non-null   datetime64[ns]
 10  appVersion            8550 non-null   object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 945.4+ KB


In [33]:
# Download resource NLTK (hanya perlu sekali)
try:
    stopwords.words('indonesian')
except LookupError:
    nltk.download('stopwords')
    nltk.download('punkt')

In [34]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [36]:
# --- DEFINISIKAN FUNGSI PREPROCESSING ---
def preprocess_text(text):
    # 1. Case Folding: Mengubah teks menjadi huruf kecil
    text = text.lower()

    # 2. Punctuation & Number Removal: Menghapus karakter, angka, dan spasi berlebih
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # 3. Tokenization: Memecah kalimat menjadi token (kata)
    tokens = word_tokenize(text)

    # 4. Stopword Removal: Menghapus kata-kata umum
    stop_words = set(stopwords.words('indonesian'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # 5. Stemming: Mengubah kata ke bentuk dasarnya
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]

    # Menggabungkan kembali token menjadi kalimat
    return ' '.join(stemmed_tokens)

# Menerapkan fungsi preprocess_text pada setiap baris di kolom 'content'
# Ini mungkin memakan waktu beberapa saat tergantung ukuran data dan kecepatan komputer Anda
print("Memulai proses pembersihan teks...")
df_reviews['clean_content'] = df_reviews['content'].apply(preprocess_text)
print("Proses pembersihan teks selesai.")

# Menampilkan 5 baris pertama dari kolom asli dan kolom yang sudah dibersihkan untuk perbandingan
print("\nBerikut perbandingan hasilnya:")
print(df_reviews[['content', 'clean_content']].head())

Memulai proses pembersihan teks...


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
